In [2]:
using PastaQ
using ITensors
using Random
using Printf
using OptimKit
using Zygote

In [3]:
function hamiltonian(N)
    os = OpSum()
  
    os += -J, "Z", 1, "Z", 2
    os += -J, "X", 1, "X", 2
    
    return os
end

function hamiltonian_ancillas(N)
    os = OpSum()
  
    os += -J, "Z", 1, "Z", 3
    os += -J, "X", 1, "X", 3
    
    return os
end

hamiltonian_ancillas (generic function with 1 method)

In [4]:
N = 3   # number of qubits
J = 1.0  # Ising exchange interaction
h = 1.  # transverse magnetic field

# Hilbert space
hilbert = qubits(N)

# define the Hamiltonian
os = hamiltonian(N)
os_ancillas = hamiltonian_ancillas(N)

# build MPO "cost function"
H = MPO(os, hilbert)
H2 = MPO(os_ancillas, hilbert)

# find ground state with DMRG

nsweeps = 100
maxdims = [10, 20, 30, 50, 100]
cutoff_ = 1e-10

start_mps = randomMPS(hilbert, linkdims=10)
Edmrg, Φ = dmrg(H2, start_mps; outputlevel=0, nsweeps, maxdims, cutoff_);
@printf("\nGround state energy from DMRG: %.10f\n\n", Edmrg)


Ground state energy from DMRG: -2.0000000000



In [5]:
full = contract(Φ.data)
full.tensor

Dim 1: (dim=2|id=61|"Qubit,Site,n=1")
Dim 2: (dim=2|id=240|"Qubit,Site,n=2")
Dim 3: (dim=2|id=995|"Qubit,Site,n=3")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2×2
[:, :, 1] =
 -0.6481410476285797     -0.2826891974924541
  1.699693154267482e-17   4.654229393228718e-16

[:, :, 2] =
 -9.77658610126822e-16   1.9511105818827065e-16
 -0.6481410476285778    -0.28268919749245586

In [6]:
full.tensor[:, 2, :]

Dim 1: 2
Dim 2: 2
NDTensors.Dense{Float64, Base.ReshapedArray{Float64, 1, SubArray{Float64, 2, Array{Float64, 3}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64, Base.Slice{Base.OneTo{Int64}}}, false}, Tuple{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64}}}}
 2×2
 -0.2826891974924541      1.9511105818827065e-16
  4.654229393228718e-16  -0.28268919749245586

In [7]:
# layer of single-qubit Ry gates
Rylayer(N, θ) = [("Ry", j, (θ=θ[j],)) for j in 1:N]

# brick-layer of CX gates
function CXlayer(N, Π)
  start = isodd(Π) ? 1 : 2
  return [("CX", (j, j + 1)) for j in start:2:(N - 1)]
end

# variational ansatz
function variationalcircuit(N, depth, θ)
  circuit = Tuple[]
  for d in 1:depth
    circuit = vcat(circuit, CXlayer(N, d))
    circuit = vcat(circuit, Rylayer(N, θ[d]))
  end
  return circuit
end

variationalcircuit (generic function with 1 method)

In [8]:
function get_bit_string(n)
    b = Matrix{Int}(undef, n, 2^n)
    for i = 0:2^(n)-1
        s = bitstring(i)
        b[:, i+1] .= [Int(i)-48 for i in s[length(s)-n+1:end]]
    end
    return b
end
get_bit_string(2)

2×4 Matrix{Int64}:
 0  0  1  1
 0  1  0  1

In [9]:
ψs = Vector{MPS}(undef, 4)
states = get_bit_string(2)
for i in 1:size(states, 2)
    state = zeros(Int, 3)
    state[[1, 3]] = states[:, i]
    #state = states[:, i]
    ψs[i] = productstate(hilbert, state)
    println(state)
end

[0, 0, 0]
[0, 0, 1]
[1, 0, 0]
[1, 0, 1]


In [10]:
depth = 10
ψ = productstate(hilbert)

cutoffs = 1e-8
maxdims = 50

# cost function
function loss(θ)
  circuit = variationalcircuit(N, depth, θ)
  Uψ = runcircuit(ψ, circuit; cutoffs, maxdims)
  return inner(Uψ', H, Uψ; cutoffs, maxdims)
end

# cost function

function loss_stat(θ)
    circuit = variationalcircuit(N, depth, θ)
    
    E = 0
    for i in 1:length(ψs)
        Uψ = runcircuit(ψs[i], circuit; cutoffs, maxdims)
        E += inner(Uψ', H, Uψ; cutoffs, maxdims)
    end
    
    return E / length(ψs)
end

function loss(θ, ψ)
  circuit = variationalcircuit(N, depth, θ)
  Uψ = runcircuit(ψ, circuit; cutoffs, maxdims)
  return inner(Uψ', H, Uψ; cutoffs, maxdims)
end

Random.seed!(1234)

# initialize parameters
θ₀ = [2π .* rand(N) for _ in 1:depth]

# run VQE using BFGS optimization
optimizer = LBFGS(; maxiter=50, verbosity=2)
function loss_and_grad(x)
  y, (∇,) = withgradient(loss_stat, x)
    display(y)
  return y, ∇
end

loss_and_grad (generic function with 1 method)

In [11]:
θ, fs, gs, niter, normgradhistory = optimize(loss_and_grad, θ₀, optimizer)
@printf("Relative error: %.3E", abs(Edmrg - fs[end]) / abs(Edmrg))

-0.24634870646683624

┌ Info: LBFGS: initializing with f = -0.246348706467, ‖∇f‖ = 7.0745e-01
└ @ OptimKit /Users/alcalde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:35


-0.8693064170989728

┌ Info: LBFGS: iter    1: f = -0.869306417099, ‖∇f‖ = 5.0313e-01, α = 1.00e+00, m = 0, nfg = 1
└ @ OptimKit /Users/alcalde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:76


-0.44447745646840453

...

-0.9999999999999984

┌ Info: LBFGS: iter   23: f = -1.000000000000, ‖∇f‖ = 1.7095e-08, α = 1.00e+00, m = 8, nfg = 1
└ @ OptimKit /Users/alcalde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:76


-1.0000000000000013

┌ Info: LBFGS: converged after 24 iterations: f = -1.000000000000, ‖∇f‖ = 3.0527e-09
└ @ OptimKit /Users/alcalde/.julia/packages/OptimKit/xpmbV/src/lbfgs.jl:138


Relative error: 5.000E-01

In [415]:
ψ = productstate(hilbert, [0, 0, 0])
loss(θ, ψ)

-1.2171279084094804

In [ ]:
new_ψs = Vector{MPS}(undef, 4)
for i in 1:size(states, 2)
    state = zeros(Int, 3)
    state[[1, 3]] = states[:, i]
    #state = states[:, i]
    ψs[i] = productstate(hilbert, state)
    println(state)
end

In [416]:
circuit = variationalcircuit(N, depth, θ)
Uψ = runcircuit(ψ, circuit; cutoffs, maxdims)
o = contract(Uψ.data)
o.tensor[:, 2, :]

Dim 1: 2
Dim 2: 2
NDTensors.Dense{Float64, Base.ReshapedArray{Float64, 1, SubArray{Float64, 2, Array{Float64, 3}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64, Base.Slice{Base.OneTo{Int64}}}, false}, Tuple{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64}}}}
 2×2
  0.3038642171657574   0.10382010356144811
 -0.16069076495493637  0.419305237121601

## Measurement

In [431]:
hilbert = qubits(1)
mps = productstate(hilbert, [1])

MPS
[1] ((dim=2|id=995|"Qubit,Site,n=1"),)


In [474]:
N = 1
ntrial = 1
gates = randomcircuit(N; depth=0, layered=false)

U = runcircuit(N, gates; process=true)

bases = randombases(N, ntrial)
preps = randompreparations(N, ntrial)



ITensor[ITensor ord=1
Dim 1: (dim=2|id=379|"Qubit,Site,n=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2-element
 0.0
 1.0]


In [483]:
U.data

1-element Vector{ITensor}:
 ITensor ord=2
Dim 1: (dim=2|id=379|"Qubit,Site,n=1")'
Dim 2: (dim=2|id=379|"Qubit,Site,n=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 1.0  0.0
 0.0  1.0

In [481]:
for n in 1:ntrial
    mgates = PastaQ.measurementgates(bases[n, :])
    ψ_in = productstate(N, preps[n, :])
    ψ_out = runcircuit(ψ_in, gates)

    Ψ_out = PastaQ.projectchannel(U, preps[n, :])
    println(Ψ_out.data)

    ψ_m = runcircuit(ψ_out, mgates)
    Ψ_m = runcircuit(Ψ_out, mgates)
end

ITensor[ITensor ord=1
Dim 1: (dim=2|id=379|"Qubit,Site,n=1")
NDTensors.Dense{Float64, Vector{Float64}}
 2-element
 0.0
 1.0]


In [445]:
typeof(mgates)

Vector{Tuple} (alias for Array{Tuple, 1})

In [449]:
mgates = PastaQ.measurementgates(["Z"])

ψ_m = runcircuit(mps, mgates)

PastaQ.array(ψ_m)

No documentation found.

`PastaQ.projectchannel` is a `Function`.

```
# 1 method for generic function "projectchannel":
[1] projectchannel(M::Union{ITensor, MPO}, prep::AbstractArray) in PastaQ at /home/leinad/.julia/packages/PastaQ/D5CCg/src/circuits/getsamples.jl:353
```
